In [23]:
import requests
import pandas as pd

In [26]:
df_trafik = pd.read_csv('data/trafiktaelling.csv')

df_trafik

,FID,t_nr,vejnavn,beskrivelse,husnummer,type,taelling_type,ktj_7_19,tung_pct,aadt_koretojer,...,aar,link,tung_pct_real,ladcykler_pct_real,lastbiler_7_19,lastbiler_pct,elloebehjul_7_19,elloebehjul_pct,andre_busser_7_19,wkb_geometry
0,trafiktaelling.1,1,H.C. Andersens Boulevard,sydøst for Jarmers Plads,4 eller 5A,3.0,Andre faste tællinger,42650.0,"3,8",49900.0,...,2016,http://kk.sites.itera.dk/apps/kk_trafik/pdf/UJ...,3.8,1.2,1016.0,NaN,NaN,NaN,NaN,POINT (12.565491397230991 55.67796084860272)
1,trafiktaelling.2,3,Bredgade,nord for Kongens Nytorv,3 eller 4,3.0,Andre faste tællinger,8984.0,"1,8",10500.0,...,2020,https://kk.sites.itera.dk/apps/kk_trafik/pdf/M...,1.8,3.3,18.0,"0,2",NaN,NaN,NaN,POINT (12.587600106423727 55.68116360238515)
2,trafiktaelling.3,4,Christian d. IV's Bro,syd for Niels Juels Gade,ingen,3.0,Andre faste tællinger,11701.0,"0,2",13700.0,...,2020,https://kk.sites.itera.dk/apps/kk_trafik/pdf/n...,0.2,2.8,NaN,NaN,NaN,NaN,NaN,POINT (12.587161874224892 55.67576459605928)
3,trafiktaelling.4,5,Dronning Louises Bro,øst for Nørrebrogade,NaN,2.0,Søsnittet,NaN,NaN,NaN,...,2020,https://kk.sites.itera.dk/apps/kk_trafik/pdf/s...,NaN,3.7,NaN,NaN,438.0,"1,4",NaN,POINT (12.563784830098982 55.68673054140091)
4,trafiktaelling.5,6,Kampmannsgade,ud for søerne,NaN,2.0,Søsnittet,3938.0,"2,2",4600.0,...,2020,https://kk.sites.itera.dk/apps/kk_trafik/pdf/b...,2.2,2.8,29.0,"34,1",NaN,NaN,6.0,POINT (12.557917540101617 55.67797681383055)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,trafiktaelling.1073,10195,Sydhavnsgade-Borgmester Christiansens Gade-Fre...,Krydstælling,NaN,101.0,Krydstællinger,NaN,NaN,NaN,...,2019,https://kk.sites.itera.dk/apps/kk_trafik/pdf/D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (12.542233637720141 55.64665981581368)
1073,trafiktaelling.1074,10196,Vejlands Allé-Artillerivej-Selinevej,Krydstælling,NaN,101.0,Krydstællinger,NaN,NaN,NaN,...,2019,https://kk.sites.itera.dk/apps/kk_trafik/pdf/5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (12.563594727350983 55.641493677099774)
1074,trafiktaelling.1075,10197,Vejlands Allé-Center Boulvard,Krydstælling,NaN,101.0,Krydstællinger,NaN,NaN,NaN,...,2018,https://kk.sites.itera.dk/apps/kk_trafik/pdf/m...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (12.575944159210048 55.640978432494656)
1075,trafiktaelling.1076,10198,Vejlands Allé-Motorvejen_vestlig fletning,Krydstælling,NaN,101.0,Krydstællinger,NaN,NaN,NaN,...,2019,https://kk.sites.itera.dk/apps/kk_trafik/pdf/E...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (12.553338532609525 55.641023522185684)


In [9]:
#%% STEP 1: REQUEST DATA FROM DMI's API
import requests
import pandas as pd
import datetime as dt

api_key = '4017479a-aed9-4909-aaac-0cb6169d7743' # Insert you own api-key from DMI between the ''. To obtain your api-key, visit url: https://confluence.govcloud.dk/display/FDAPI/Danish+Meteorological+Institute+-+Open+Data Then choose to get an API-key for the metObsAPI 2.0.

url = 'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items' # DMI API v2 url


# Insert local start time and UTC time shift:
#                        yyyy-mm-ddThh:mm:ss+hh:mm UTC time shift] (2 h = DK summer time, 1 h = DK winter or normal time)
from_time =             '2021-11-29T00:00:00+02:00' 

# Insert local end time and UTC time shift:
#                        yyyy-mm-ddThh:mm:ss+hh:mm UTC time shift] (2 h = DK summer time, 1 h = DK winter or normal time)
to_time =               '2021-11-30T23:59:59+02:00'

# DMI's API retuns data in UTC time - not nice when you want to compare with your Danish local time. We'll need to correct for this later on. Here we just enter the season dependent time shift:
UTC_to_local_time_shift = 2 # unit = hours. Enter local time shift relative to UTC time: I.e., enter '2' h = DK summer time, or enter '1' h = DK winter or normal time

fromtotime = from_time + '/' + to_time # concatenates from and to times 

# Specify parameter:
parm = 'pressure_at_sea' # see parameterId-list at url: https://confluence.govcloud.dk/pages/viewpage.action?pageId=26476616

# Specify station no.:
stat = '06184' # see stationId-list at url: https://confluence.govcloud.dk/pages/viewpage.action?pageId=41717704#Stations(metObs)-StationsavailableforthemetObsservice

# Specifies query:
params = {'api-key' : api_key,
          'datetime' : fromtotime,
          'stationId' : stat,
          'parameterId' : parm,  
          'limit' : '300000', # check later, that the DataFrame "data" contains less records than set limit!
          }

r = requests.get(url, params=params) # when run, this returns a "response object" of the reqest module

json = r.json() # extracts JSON data from "response object". JSON is a human-readable format for data exchange.

from pandas.io.json import json_normalize # The json object has a nested structure (e.g., dictionaries being contained in dictionaries)... 
df = json_normalize(json['features']) # converts the JSON object to a Pandas DataFrame. Using json_normalize() we select from the json object only the dictionary named 'features' and use these as columns headers in the final DataFrame


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [10]:
df

""


In [11]:
# (lon, lat) 12.447848326393638, 55.606375666851676
# 55.73107930538074, 12.692825590170797

url = 'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items' 

header={'X-Gravitee-Api-Key': api_key} #hidden 

# Parameters: https://confluence.govcloud.dk/pages/viewpage.action?pageId=26476616
param_list = ['temp_mean_past1h', 'wind_dir_past1h','wind_speed_past1h','precip_past1h', 'weather']

#Construct weather dataframe
df_weather = pd.DataFrame()

for param in param_list:

    # Specify query parameters
    params = {'datetime' : '2005-01-01T00:00:00Z/2014-12-31T23:59:00Z',
            'parameterId' : param,
            'limit' : '300000',
            # Use bbox from above
            'bbox' : '12.447848326393638,55.606375666851676,12.692825590170797,55.73107930538074'
            }


    r = requests.get(url,headers=header,params=params) # submit GET request based on url and headers
    json_t = r.json()['features']
    df_weather_t = pd.json_normalize(json_t)
    df_weather = pd.concat([df_weather, df_weather_t])

<Response [200]> https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?datetime=2005-01-01T00%3A00%3A00Z%2F2014-12-31T23%3A59%3A00Z&parameterId=temp_mean_past1h&limit=300000&bbox=12.447848326393638%2C55.606375666851676%2C12.692825590170797%2C55.73107930538074
<Response [200]> https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?datetime=2005-01-01T00%3A00%3A00Z%2F2014-12-31T23%3A59%3A00Z&parameterId=wind_dir_past1h&limit=300000&bbox=12.447848326393638%2C55.606375666851676%2C12.692825590170797%2C55.73107930538074
<Response [200]> https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?datetime=2005-01-01T00%3A00%3A00Z%2F2014-12-31T23%3A59%3A00Z&parameterId=wind_speed_past1h&limit=300000&bbox=12.447848326393638%2C55.606375666851676%2C12.692825590170797%2C55.73107930538074
<Response [200]> https://dmigw.govcloud.dk/v2/metObs/collections/observation/items?datetime=2005-01-01T00%3A00%3A00Z%2F2014-12-31T23%3A59%3A00Z&parameterId=precip_past1h&limit=300000&b

In [17]:
df_weather['properties.parameterId'].unique()

array(['temp_mean_past1h', 'wind_dir_past1h', 'wind_speed_past1h',
       'precip_past1h', 'weather'], dtype=object)

In [18]:
df_weather

,id,type,geometry.coordinates,geometry.type,properties.created,properties.observed,properties.parameterId,properties.stationId,properties.value
0,4debab73-a20f-e3e3-b2bf-0d3571f1c03a,Feature,"[12.6455, 55.614]",Point,2022-02-23T04:21:36.478108Z,2014-12-31T23:00:00Z,temp_mean_past1h,06180,5.0
1,2311654c-ce3f-bcb5-eefc-d2c6024e742b,Feature,"[12.6455, 55.614]",Point,2022-02-23T04:21:36.334375Z,2014-12-31T22:00:00Z,temp_mean_past1h,06180,5.2
2,847bb3a8-5880-d74f-5356-c46e59223e24,Feature,"[12.6455, 55.614]",Point,2022-02-23T04:21:36.148388Z,2014-12-31T21:00:00Z,temp_mean_past1h,06180,5.5
3,da1c8a09-3fdb-1431-e210-f3c5eaf4c9cd,Feature,"[12.6455, 55.614]",Point,2022-02-23T04:21:35.978337Z,2014-12-31T20:00:00Z,temp_mean_past1h,06180,5.8
4,7e60a495-cefc-ccbb-8cfd-2239ed6714ed,Feature,"[12.6455, 55.614]",Point,2022-02-23T04:21:35.831114Z,2014-12-31T19:00:00Z,temp_mean_past1h,06180,5.8
...,...,...,...,...,...,...,...,...,...
16301,37696507-a1ca-2796-c695-4b42f936a51c,Feature,"[12.6455, 55.614]",Point,2022-02-23T09:33:02.397711Z,2005-01-01T12:00:00Z,weather,06180,10.0
16302,c0f91da4-99ef-7eea-4853-26f9c0f99766,Feature,"[12.6455, 55.614]",Point,2022-02-23T09:33:02.394710Z,2005-01-01T09:00:00Z,weather,06180,10.0
16303,c71b653c-84d9-dfc0-34b1-005c4a5c3b9c,Feature,"[12.6455, 55.614]",Point,2022-02-23T09:33:02.391423Z,2005-01-01T06:00:00Z,weather,06180,2.0
16304,166379dd-2aa4-8914-20d2-9aafdea39d36,Feature,"[12.6455, 55.614]",Point,2022-02-23T09:33:02.388680Z,2005-01-01T03:00:00Z,weather,06180,46.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=61ec0a5b-e327-402f-95c1-91c4aeb91c25' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>